In [ ]:
import shutil
import os
import urllib

# Get url from https://www.worldcubeassociation.org/export/results
url = input()

# If url contains .sql, replace with .tsv
url = url.replace('.sql', '.tsv')

print("Download")
urllib.request.urlretrieve(url, "WCA_export.zip")

if os.path.exists('WCA_export'):
    shutil.rmtree('WCA_export')

print("Unzip")
shutil.unpack_archive('WCA_export.zip', 'WCA_export')

print("Rename")
for filename in os.listdir('WCA_export'):
    # Remove WCA_export_ from the filename
    new_name = filename.replace('WCA_export_', '')
    os.rename(f'WCA_export/{filename}', f'WCA_export/{new_name}')

print("Remove unnecessary")
os.remove('./WCA_export/championships.tsv')
os.remove('./WCA_export/eligible_country_iso2s_for_championship.tsv')
os.remove('./WCA_export/Formats.tsv')
os.remove('./WCA_export/RoundTypes.tsv')
os.remove('./WCA_export/Scrambles.tsv')

In [ ]:
import pandas as pd
import os
import time

export_dir = 'WCA_export'

start = time.time()

filenames = os.listdir(export_dir)
filenames = [f for f in filenames if f.endswith('.tsv')]
tablenames = [filename.split('.')[0] for filename in filenames]

dfs = {}

for filename in filenames:
    tablename = filename.split('.')[0]
    dfs[tablename] = pd.read_csv(f'{export_dir}/{filename}', delimiter='\t')
    print(filename)

In [ ]:
print('Remove unnecessary columns')
dfs['Persons'] = dfs['Persons'].drop(columns=['subid'])
dfs['Results'] = dfs['Results'].drop(columns=['personName', 'formatId', 'value1', 'value2', 'value3', 'value4', 'value5', 'personCountryId'])
dfs['Results'].head()

In [ ]:
print('Cast event ids to string')
dfs['Events'].id = dfs['Events'].id.astype(str)
dfs['RanksSingle'].eventId = dfs['RanksSingle'].eventId.astype(str)
dfs['RanksAverage'].eventId = dfs['RanksAverage'].eventId.astype(str)

In [ ]:
print('Handle duplicate persons')

# If a person has moved countries, then they could have multiple entries
dfs['Persons'].drop_duplicates('id', inplace=True)

In [ ]:
print('Populate country ids')
country_ids = dfs['Persons'][['id', 'countryId']]

dfs['RanksSingle'] = dfs['RanksSingle'].merge(country_ids, left_on='personId', right_on='id').drop('id', axis=1)
dfs['RanksAverage'] = dfs['RanksAverage'].merge(country_ids, left_on='personId', right_on='id').drop('id', axis=1)
dfs['RanksSingle']

In [ ]:
print('Populate continent ids')
continent_ids = dfs['Countries'][['id', 'continentId']]

dfs['RanksSingle'] = dfs['RanksSingle'].merge(continent_ids, left_on='countryId', right_on='id').drop('id', axis=1)
dfs['RanksAverage'] = dfs['RanksAverage'].merge(continent_ids, left_on='countryId', right_on='id').drop('id', axis=1)
dfs['Persons'] = dfs['Persons'].merge(continent_ids, left_on='countryId', right_on='id', suffixes=('', '_drop')).drop('id_drop', axis=1)

In [ ]:
print('Populate names')
names = dfs['Persons'][['id', 'name']]

dfs['RanksSingle'] = dfs['RanksSingle'].merge(names, left_on='personId', right_on='id').drop('id', axis=1)
dfs['RanksAverage'] = dfs['RanksAverage'].merge(names, left_on='personId', right_on='id').drop('id', axis=1)

In [ ]:
print('Calculate max ranks')
single_world_maxes = dfs['RanksSingle'].groupby('eventId').max()['worldRank']
average_world_maxes = dfs['RanksAverage'].groupby('eventId').max()['worldRank']

single_continent_maxes = dfs['RanksSingle'].groupby(['continentId', 'eventId']).max()['continentRank']
average_continent_maxes = dfs['RanksAverage'].groupby(['continentId', 'eventId']).max()['continentRank']

single_country_maxes = dfs['RanksSingle'].groupby(['countryId', 'eventId']).max()['countryRank']
average_country_maxes = dfs['RanksAverage'].groupby(['countryId', 'eventId']).max()['countryRank']

In [10]:
import itertools

def calculate_sum_of_ranks(type, events, world_maxes, continent_maxes, country_maxes):

    person_ids = dfs['Persons'].id.unique()
    combinations = pd.DataFrame(list(itertools.product(person_ids, events)), columns=['personId', 'eventId'])

    # Make a row for every person and every event. If person has no result, their rank is NaN
    ranks_all = combinations \
        .merge(dfs[f'Ranks{type}'][['personId', 'eventId', 'worldRank', 'continentRank', 'countryRank']], on=['personId', 'eventId'], how='left') \
        .merge(dfs['Persons'][['id', 'countryId', 'continentId']], left_on='personId', right_on='id', how='left')

    ranks_all['worldRank'] = ranks_all['worldRank'].fillna(ranks_all['eventId'].map(world_maxes))
    ranks_all['continentRank'] = ranks_all['continentRank'].fillna(ranks_all.set_index(['continentId', 'eventId']).index.map(continent_maxes).to_series(index=ranks_all.index))
    ranks_all['countryRank'] = ranks_all['countryRank'].fillna(ranks_all.set_index(['countryId', 'eventId']).index.map(country_maxes).to_series(index=ranks_all.index))

    sor = ranks_all.groupby('personId')[['worldRank', 'continentRank', 'countryRank']].sum()
    sor = sor.reset_index()
    sor = sor.rename(columns={
        'worldRank': f'worldSor{type}',
        'continentRank': f'continentSor{type}',
        'countryRank': f'countrySor{type}',
    })
    return sor

In [ ]:
single_events = ['222', '333', '333bf', '333fm', '333mbf', '333oh', '444', '444bf', '555', '555bf', '666', '777', 'clock', 'minx', 'pyram', 'skewb', 'sq1']

# Same as single_events but no 333mbf
average_events = ['222', '333', '333bf', '333fm', '333oh', '444', '444bf', '555', '555bf', '666', '777', 'clock', 'minx', 'pyram', 'skewb', 'sq1']

print('Single sum of rannks')
single_sor = calculate_sum_of_ranks('Single', single_events, single_world_maxes, single_continent_maxes, single_country_maxes)
print('Average sum of ranks')
average_sor = calculate_sum_of_ranks('Average', average_events, average_world_maxes, average_continent_maxes, average_country_maxes)

In [ ]:
dfs['Persons'] = dfs['Persons'].merge(single_sor, left_on='id', right_on='personId').drop('personId', axis=1)
dfs['Persons'] = dfs['Persons'].merge(average_sor, left_on='id', right_on='personId').drop('personId', axis=1)
dfs['Persons'].head()

In [ ]:
best_singles = dfs['RanksSingle'][dfs['RanksSingle'].worldRank == 1][['eventId', 'best']].rename(columns={'best': 'single'})
best_averages = dfs['RanksAverage'][dfs['RanksAverage'].worldRank == 1][['eventId', 'best']].rename(columns={'best': 'average'})

world_bests = best_singles.merge(best_averages, on='eventId', how='outer')
world_bests = {eventId: (single, average) for eventId, single, average in world_bests.values}
world_bests

In [14]:
def group_dict(d):
    '''
    Example input: {
        ('a', 'b'): 1,
        ('a', 'c'): 2,
    }

    Example output: {
        'a': {
            'b': 1,
            'c': 2,
        },
    }
    '''
    output = {}
    for k in d:
        if k[0] not in output:
            output[k[0]] = {}
        
        output[k[0]][k[1]] = d[k]
    return output

best_singles = dfs['RanksSingle'][dfs['RanksSingle'].continentRank == 1][['eventId', 'best', 'continentId']].rename(columns={'best': 'single'})
best_averages = dfs['RanksAverage'][dfs['RanksAverage'].continentRank == 1][['eventId', 'best', 'continentId']].rename(columns={'best': 'average'})

continent_bests = best_singles.merge(best_averages, on=['eventId', 'continentId'], how='outer').drop_duplicates()
continent_bests = {(continentId, eventId): (single, average) for eventId, single, continentId, average in continent_bests.values}
continent_bests = group_dict(continent_bests)

best_singles = dfs['RanksSingle'][dfs['RanksSingle'].countryRank == 1][['eventId', 'best', 'countryId']].rename(columns={'best': 'single'})
best_averages = dfs['RanksAverage'][dfs['RanksAverage'].countryRank == 1][['eventId', 'best', 'countryId']].rename(columns={'best': 'average'})

country_bests = best_singles.merge(best_averages, on=['eventId', 'countryId'], how='outer').drop_duplicates()
country_bests = {(continentId, eventId): (single, average) for eventId, single, continentId, average in country_bests.values}
country_bests = group_dict(country_bests)

In [ ]:
def build_rank_dict(tablename):
    d = {}
    for row in dfs[tablename][['personId', 'eventId', 'best']].values:
        personId, eventId, best = row

        if personId not in d:
            d[personId] = {}

        d[personId][eventId] = best
    
    # Make sure every person at least has an empty object
    for id in dfs['Persons']['id']:
        if id not in d:
            d[id] = {}

    return d

print('Build rank dicts')
single_dict = build_rank_dict('RanksSingle')
average_dict = build_rank_dict('RanksAverage')

In [16]:
import math

def mbldScore(value):
    if not value:
        return 0
    seconds = math.floor(value / 100) % 1e5
    points = 99 - (math.floor(value / 1e7) % 100)
    centiseconds = None if seconds == 99999 else seconds * 100
    proportionOfHourLeft = 1 - centiseconds / 360000
    score = points + proportionOfHourLeft
    return max(score, 0)

def get_kinch_score(personId, bests, key):
    if key:
        if key in bests:
            bests = bests[key]
        else:
            # This edge case can occur if, for example, a person moves to a new country that has no results.
            # This occurred for wca id 2018YEDD01 who moved to Barbados!
            return 0
    scores = []

    # Handle 333mbf
    single = single_dict[personId].get("333mbf")
    average = average_dict[personId].get("333mbf")
    bestSingle, bestAverage = bests["333mbf"] if "333mbf" in bests else (None, None)

    mbldPersonal = mbldScore(single)
    mbldRecord = mbldScore(bestSingle)

    if mbldRecord:
        scores.append(mbldPersonal / mbldRecord * 100)
    else:
        # If nobody has mbld, use 100
        scores.append(0)

    # For these events, use better between single and average
    for eventId in ["333fm", "333bf", "444bf", "555bf"]:
        single = single_dict[personId].get(eventId)
        average = average_dict[personId].get(eventId)
        bestSingle, bestAverage = bests[eventId] if eventId in bests else (None, None)

        if not single and not average:
            scores.append(0)
        elif not bestSingle or not bestAverage:
            # This can happen if a person has multiple countryIds and one of the countries has no result for the event.
            scores.append(100)
        elif not average:
            # If no average, use single
            scores.append(bestSingle / single * 100)
        else:
            # If there is an average, use the better of the two
            scores.append(max(
                bestSingle / single * 100,
                bestAverage / average * 100
            ))

    # For these events, use average
    for eventId in ['222', '333', '333oh', '444', '555', '666', '777', 'clock', 'minx', 'pyram', 'skewb', 'sq1']:
        single = single_dict[personId].get(eventId)
        average = average_dict[personId].get(eventId)
        bestSingle, bestAverage = bests[eventId] if eventId in bests else (None, None)

        if not average:
            scores.append(0)
        elif not bestAverage:
            # This can happen if a person has multiple countryIds and one of the countries has no result for the event.
            scores.append(100)
        else:
            scores.append(bestAverage / average * 100)

    avgScore = sum(scores) / len(scores)
    return avgScore

In [ ]:
persons = dfs['Persons']

print('World kinch')
persons['worldKinch'] = persons.apply(lambda row: get_kinch_score(row['id'], world_bests, None), axis=1)

print('Continent kinch')
persons['continentKinch'] = persons.apply(lambda row: get_kinch_score(row['id'], continent_bests, row['continentId']), axis=1)

print('Country kinch')
persons['countryKinch'] = persons.apply(lambda row: get_kinch_score(row['id'], country_bests, row['countryId']), axis=1)

dfs['Persons'].head()

In [ ]:
print('Populate startDate and endDate')

# Format is yyyy-mm-dd
# Pad month and day with zeros
dfs['Competitions']['startDate'] = dfs['Competitions'].apply(lambda row: f"{row['year']}-{str(row['month']).zfill(2)}-{str(row['day']).zfill(2)}", axis=1)
dfs['Competitions']['endDate'] = dfs['Competitions'].apply(lambda row: f"{row['year']}-{str(row['endMonth']).zfill(2)}-{str(row['endDay']).zfill(2)}", axis=1)

In [ ]:
print('Calculate birthdays')

# Get competition data
comps = dfs['Results'][['competitionId', 'personId']].drop_duplicates()
comps = comps.merge(dfs['Competitions'][['id', 'startDate']], left_on='competitionId', right_on='id').drop('id', axis=1)
comps = comps.merge(dfs['Persons'][['id', 'name']], left_on='personId', right_on='id').drop('id', axis=1)
comps = comps.sort_values('startDate')

# Get first comp for each person
first_comps = {}
for row in comps.values:
    personId = row[1]

    if personId in first_comps:
        continue

    first_comps[personId] = row

def sort_dict(d, keys):
    output = {}
    for key in keys:
        output[key] = d[key]
    return output

# Get persons in order of rank
persons = dfs['RanksSingle'].sort_values('worldRank')['personId'].unique()

first_comps = sort_dict(first_comps, persons)

dfs['Birthdays'] = pd.DataFrame(first_comps.values(), columns=['competitionId', 'personId', 'date', 'name'])
dfs['Birthdays'].head()

In [ ]:
import json

with open(f'{export_dir}/metadata.json', 'r') as f:
    data = json.loads(f.read())

    dfs['Miscellaneous'] = pd.DataFrame({
        'key': ['export_date'],
        'value': [data['export_date'][0:10]], # Only first 10 chars for yyyy-mm-dd
    })

dfs['Miscellaneous']

In [ ]:
import sqlite3

def df_to_sqlite(df, table_name):
    conn = sqlite3.connect('wca.db')

    try:
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        print(f"{table_name} table created")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        conn.close()

# Remove wca.db
if os.path.exists('wca.db'):
    os.remove('wca.db')

for name in dfs:
    df_to_sqlite(dfs[name], name)

In [ ]:
print('Create indices')
conn = sqlite3.connect('wca.db')
c = conn.cursor()

# TODO: Analyze which of these indices are actually needed
c.execute('CREATE INDEX idx_Persons_id ON Persons(id);')
c.execute('CREATE INDEX idx_Persons_countryId ON Persons(countryId);')
c.execute('CREATE INDEX idx_Persons_continentId ON Persons(continentId);')
c.execute('CREATE INDEX idx_Persons_countryKinch ON Persons(countryKinch);')
c.execute('CREATE INDEX idx_Persons_continentKinch ON Persons(continentKinch);')
c.execute('CREATE INDEX idx_Persons_worldKinch ON Persons(worldKinch);')
c.execute('CREATE INDEX idx_Persons_countrySorSingle ON Persons(countrySorSingle);')
c.execute('CREATE INDEX idx_Persons_continentSorSingle ON Persons(continentSorSingle);')
c.execute('CREATE INDEX idx_Persons_worldSorSingle ON Persons(worldSorSingle);')
c.execute('CREATE INDEX idx_Persons_countrySorAverage ON Persons(countrySorAverage);')
c.execute('CREATE INDEX idx_Persons_continentSorAverage ON Persons(continentSorAverage);')
c.execute('CREATE INDEX idx_Persons_worldSorAverage ON Persons(worldSorAverage);')
c.execute('CREATE INDEX idx_RanksSingle_eventId ON RanksSingle(eventId);')
c.execute('CREATE INDEX idx_RanksSingle_personId ON RanksSingle(personId);')
c.execute('CREATE INDEX idx_RanksSingle_worldRank ON RanksSingle(worldRank);')
c.execute('CREATE INDEX idx_RanksSingle_continentRank ON RanksSingle(continentRank);')
c.execute('CREATE INDEX idx_RanksSingle_countryRank ON RanksSingle(countryRank);')
c.execute('CREATE INDEX idx_RanksAverage_eventId ON RanksAverage(eventId);')
c.execute('CREATE INDEX idx_RanksAverage_personId ON RanksAverage(personId);')
c.execute('CREATE INDEX idx_RanksAverage_worldRank ON RanksAverage(worldRank);')
c.execute('CREATE INDEX idx_RanksAverage_continentRank ON RanksAverage(continentRank);')
c.execute('CREATE INDEX idx_RanksAverage_countryRank ON RanksAverage(countryRank);')
c.execute('CREATE INDEX idx_Staff_wca_id ON Staff(wca_id);')

conn.commit()
conn.close()